# Step1. 검증용 데이터셋 준비

# Step2. Google OCR API, keras-ocr, Tesseract로 테스트 진행

# Step3. 테스트 결과 정리

# Step4. 결과 분석과 결론 제시

# 루브릭
1.OCR을 활용하여 구현하려는 서비스의 기획이 타당한가?
- 목표로 하는 서비스가 OCR를 적용 가능하며, OCR을 활용했을 때 더욱 유용해진다.

2.모델 평가기준이 명확하고 체계적으로 세워졌는가?
- 평가 기준에 부합하는 테스트 데이터의 특징이 무엇인지 명확하게 제시되었다.

3.평가기준에 따라 충분한 분량의 테스트가 진행되고 그 결과가 잘 정리되었는가?
- 최대 20장까지의 테스트 이미지를 사용해 제시된 평가 기준에 따른 테스트 결과가 잘 정리되어 결론이 도출되었다.

# 회고

